# Multiple chains

#### 1. 필수 Library 설치
LCEL을 사용하기 위한 필수 라이브러리 설치

In [27]:
!pip install --upgrade --quiet langchain langchain-openai

#### 2. OpenAI API KEY 인증

In [2]:
import os
os.environ["OPENAI_API_KEY"]      = "sk-*******************************************************"

#### 3. Multiple Chain 실행

##### 1) Chain 결과값을 활용한 Chain 연결 
Chain2(Chain1_Output, Chain2_Value)

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#Multi로 들어갈 Prompt Template 생성
prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

#OpenAI Model 불러오기
model = ChatOpenAI()

#LCEL 구조로 Prompt별 Chain 생성
#StrOutputParser() : String으로 Output 반환
chain1 = prompt1 | model | StrOutputParser()

#Chain1 의 결과값을 Value로 설정하여 받아옴.
#itemgetter : "language" 변수를 받아오도록 설정.
chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

#chain2 실행
chain2.invoke({"person": "obama", "language": "korean"})

'시카고는 일리노이 주에 위치한 도시입니다.'

##### 2) Generator 활용 
Multi Prompt를 LCEL 형태의 chain으로 만들어 generator에 연결한 후 output을 input값으로 활용.

model_parser = model에 프롬프트를 전달 

chain 1 = prompt1 | model_parser

chain 2 = prompt2 | model_parser...

generator = (chain1 | chain2 ...)

final_prompt = generator.invoke("") # input 값을 넣어서 generator 결과값 도출 

model.invoke(final_prompt) # generator 결과값을 모델에 실행

In [ ]:
from langchain_core.runnables import RunnablePassthrough

#multi prompt 생성 
prompt1 = ChatPromptTemplate.from_template(
    "generate a {attribute} color. Return the name of the color and nothing else:"
)
prompt2 = ChatPromptTemplate.from_template(
    "what is a fruit of color: {color}. Return the name of the fruit and nothing else:"
)
prompt3 = ChatPromptTemplate.from_template(
    "what is a country with a flag that has the color: {color}. Return the name of the country and nothing else:"
)
prompt4 = ChatPromptTemplate.from_template(
    "What is the color of {fruit} and the flag of {country}?"
)

#LCEL Model parser 생성
model_parser = model | StrOutputParser()

#각각의 개별적인 chain 생성
color_generator = (
    {"attribute": RunnablePassthrough()} | prompt1 | {"color": model_parser}
)
color_to_fruit = prompt2 | model_parser
color_to_country = prompt3 | model_parser

#각 chain을 generator로 연결
question_generator = (
    color_generator | {"fruit": color_to_fruit, "country": color_to_country} | prompt4
)

#generator에 input값을 넣어 최종 prompt 생성 
prompt = question_generator.invoke("warm")

#최종 prompt를 모델에 적용하여 최종 답변 도출 
model.invoke(prompt)

AIMessage(content='The color of orange is orange, and the flag of Cyprus is white with a copper-colored map of the island in the center.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

## Branching and Merging

#### LCEL 을 여러개 생성하여 하나의 Chain으로 결합

In [30]:
#입력값을 받아 base_response 변수에 저장
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

#planner의 결과값을 받아서 실행
arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

#planner의 결과 값을 받아서 실행
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

#모든 결과값을 하나의 답변으로 생성
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

#실행 순서대로 Chain 생성 
# planner 실행 - planner 값을 활용한 결과값을 지정 - final_responder에서 결과 값을 받아와 최종 실행
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

#chain 실행
chain.invoke({"input": "scrum"})

'While Scrum offers numerous benefits such as promoting collaboration, adaptability, and continuous improvement, there are indeed some potential drawbacks to consider. These include the need for a high level of commitment from team members, challenges with large or complex projects, potential undefined roles and responsibilities, scope creep risks, and dependencies on strong team dynamics.\n\nTo address these concerns and maximize the effectiveness of Scrum, it is crucial to ensure clear communication, well-defined roles, and a strong commitment from all team members. Additionally, proper project planning and monitoring can help mitigate scope creep and adapt Scrum practices to suit the specific needs of the project. By proactively addressing these potential challenges, teams can harness the power of Scrum to deliver high-quality results efficiently and effectively.'